In [ ]:
# Importing libraries and modules
import Pkg; Pkg.activate("..")
using Pkg; Pkg.instantiate()
include("MonodSDEsim.jl")
using .MonodSDEsim, Plots

### Biochemical Reaction Kinetics

Consider the following biochemical reactions

$$ 
\begin{align}
    \text{Substrate S} &\xrightarrow{\text{Biomass X}} \text{more Biomass X}, \\
    \text{Substrate S} &\xrightarrow{\text{Biomass X}} \text{Product Y}, \\
    \text{Product Y} &\xrightarrow{\text{Biomass X}} \text{Byproduct Z}.
\end{align}
$$

In [ ]:
# Setting up the model parameters
# The parameters are defined in a dictionary for easy access and modification
params  = Dict(
    "μ_max" => 0.244, 
    "K_sx" => 0.15, 
    "Y_xs" => 0.28,
    "Y_ys" => 0.42,
    "m_s" => 0.015,
    "q_max_y" => 1.79,
    "K_sy" => 1.68,
    "q_max_z" => 0.11,
    "K_sz" => 0.15,
    "σs" => 0.5, 
    "K_σs" => 0.1,
    "σx" => 0.05, 
    "K_σx" => 0.1,
    "σy" => 0.05,
    "K_σy" => 0.1, 
    "σz" => 0.05,
    "K_σz" => 0.1
)

# Initial conditions for the state variables
# The initial conditions are defined in a dictionary for easy access and modification
init = Dict(
    "S0" => 25.0, 
    "X0" => 1.0, 
    "Y0" => 0, 
    "Z0" => 0
)

# Time parameters
tspan = (0.0, 10.0)  # Time span for the simulation

sol = simulate(params, tspan, init)

# Plotting the results
plot(sol, title="Monod SDE Simulation", label=["Substrate (S)" "Biomass (X)" "Product (P)" "Byproduct (Z)"], xlabel="Time", ylabel="Concentration")
plot!(legend=:topright, fontfamily="Computer Modern", lw=2, markersize=4, grid=true, ylims=(0, init["S0"]), xlims=(0, 10),)